In [1]:
import os
import pandas as pd
import requests
import getpass
from bs4 import BeautifulSoup
import json

In [2]:
key = getpass.getpass()

········


In [3]:
def summoner_name(key, name):
    name = name.replace(' ', '%20')
    request = requests.get(f'https://br1.api.riotgames.com/tft/summoner/v1/summoners/by-name/{name}?api_key={key}')
    soup = BeautifulSoup(request.content)
    return eval(soup.text)

In [4]:
 teste = summoner_name(key, 'Es mir egal')

In [5]:
pd.DataFrame([teste])

,id,accountId,puuid,name,profileIconId,revisionDate,summonerLevel
0,4GfUXIlMw5B040aVRVjxDlHVKkhiy4DP8T_HjdeuxG-QoV8,TYPXovIvidbWnodF3_pVrS-Uz7BKg-1x_7LWyVTvrLFV7-U,K3mbUvb-2PPBom5fdP5FBdn-2CUsQqy4t2wh-dmw-e55tv...,Es mir egal,4498,1587925044000,84


In [6]:
def match_list(key, puuid, n = 30):
    request = requests.get(f'https://americas.api.riotgames.com/tft/match/v1/matches/by-puuid/{puuid}/ids?count={n}&api_key={key}')
    soup = BeautifulSoup(request.content)
    return eval(soup.text)

In [7]:
match_list(key, teste['puuid'])

['BR1_1927689028',
 'BR1_1927580618',
 'BR1_1927468990',
 'BR1_1927461391',
 'BR1_1926730533',
 'BR1_1926675398',
 'BR1_1926670564',
 'BR1_1926614700',
 'BR1_1926538340',
 'BR1_1926140313',
 'BR1_1926073659',
 'BR1_1926067196',
 'BR1_1926063272',
 'BR1_1925978166',
 'BR1_1925750238',
 'BR1_1925712925',
 'BR1_1925627632',
 'BR1_1925583474',
 'BR1_1925376688',
 'BR1_1925432251',
 'BR1_1925266933',
 'BR1_1925232809',
 'BR1_1925207099',
 'BR1_1919021209',
 'BR1_1918904380',
 'BR1_1918819128',
 'BR1_1918810872',
 'BR1_1918745440',
 'BR1_1918760191',
 'BR1_1918703027']

In [8]:
def match_info(key, matchid):
    '''Return a dataframe with some infos from the match (matchid, datetime, game lenght(s), game variation,
    game version, number of the set).
    
    key = riot API-key
    matchid = The id from a specific match of team fight tatics(TFT)'''
    
    request = requests.get(f'https://americas.api.riotgames.com/tft/match/v1/matches/{matchid}?api_key={key}')
    soup = BeautifulSoup(request.content)
    df = pd.DataFrame(pd.json_normalize(eval(soup.text)))
    df2 = df[['metadata.match_id', 'info.game_datetime', 'info.game_length', 'info.game_variation',
       'info.game_version', 'info.tft_set_number']]
    df2.columns = [i.replace('metadata.', '').replace('info.', '') for i in df2.columns]
    
    return df2

In [9]:
match_info(key, 'BR1_1926730533')

,match_id,game_datetime,game_length,game_variation,game_version,tft_set_number
0,BR1_1926730533,1587870567039,2199.608887,TFT3_GameVariation_FreeNeekos,Version 10.8.317.8137 (Apr 17 2020/16:09:11) [...,3


In [10]:
def match_participants(key, matchid):
    '''Return a list with the puuid from the 8 participants of the match'''
    
    request = requests.get(f'https://americas.api.riotgames.com/tft/match/v1/matches/{matchid}?api_key={key}')
    soup = BeautifulSoup(request.content)
    df = pd.DataFrame(pd.json_normalize(eval(soup.text)))
    return df.iloc[0,2]

In [11]:
match_participants(key, 'BR1_1926730533')

['qYjGRXS83Z7LIC4c40Hk27k2ee6SrYKWhGOwdRbz2IjXf42FEWv-dvIcisNE9lEFbPwVzkCCJ34KRQ',
 'ZOp977RUzO2rc4XXpC5LDmSW9FVPinsos54x01UvxLH98PpP3NYXmClEjBL7z1YWn1NsOFa9JXtPIg',
 'BsmmhSCiK5vK67zbbMhAz2edViDJ0dvNlTuQOZQNcbu13xPKgDvxptOsdCmg3nfAfFl4wBy2YqtVAQ',
 'Nd0RYeDsIefGnmlGA45x6zj-thg349ozYFcEobPFfROdqdBCF9Gs3SeWzIgqagknLSiDqgxEqqNUiw',
 'VZV01cG8R-A-yZ5spGtEhMTOfsdVBZhf-4EDIkcEhgjb3M8botGBmNEa64FxmNGUxiHyydTByqVWyA',
 'K3mbUvb-2PPBom5fdP5FBdn-2CUsQqy4t2wh-dmw-e55tv0nVirYpqo-AISeib2MLctqZ63zorohUg',
 'yWg9adnTaYHLhXpyOgkDilt9Oyv80_n4-RlL19ZpQtnXpOWgHRcAsXcynltTP4w9hi0xhQ9VbK14Cg',
 'OpAqJOjE1o8Y1l3cb50mh1iaUnDONPCzEZUm8fobWVot-6wBa8ltTlG6joXoyEYSqJ8ladkieMBW5g']

In [111]:
matchid = 'BR1_1926730533'


request = requests.get(f'https://americas.api.riotgames.com/tft/match/v1/matches/{matchid}?api_key={key}')
soup = BeautifulSoup(request.content)
df_master = pd.DataFrame(columns = ['match_id', 'gold_left', 'last_round', 'level', 'placement', 'players_eliminated', 'puuid',
                             'time_eliminated', 'total_damage_to_players', 'blademaster_num', 'blaster_num',
                             'brawler_num', 'celestial_num', 'chrono_num', 'cybernetic_num', 'dark_star_num',
                             'demolitionist_num', 'infiltrator_num', 'mana_reaver_num', 'mech_pilot_num',
                             'mercenary_num', 'mystic_num', 'protector_num', 'rebel_num', 'sniper_num',
                             'space_pirate_num', 'star_guardian_num', 'starship_num', 'sorcerer_num', 
                             'valkyrie_num', 'vanguard_num', 'void_num', 'champion_1', 'champion_2', 'champion_3',
                             'champion_4', 'champion_5', 'champion_6', 'champion_7', 'champion_8', 'champion_9',
                             'champion_10'])

df_match = pd.DataFrame(pd.json_normalize(eval(soup.text)))
dict_players = df_match.iloc[0,-3]

In [114]:
for i in range(8):
    info_player = pd.DataFrame([dict_players[i]])
    df_master.loc[i, 'match_id'] = df_match.loc[0, 'metadata.match_id']
    df_master.loc[i, 'gold_left'] = info_player.loc[0, 'gold_left']
    df_master.loc[i, 'last_round'] = info_player.loc[0, 'last_round']
    df_master.loc[i, 'level'] = info_player.loc[0, 'level']
    df_master.loc[i, 'placement'] = info_player.loc[0, 'placement']
    df_master.loc[i, 'players_eliminated'] = info_player.loc[0, 'players_eliminated']
    df_master.loc[i, 'puuid'] = info_player.loc[0, 'puuid']
    df_master.loc[i, 'time_eliminated'] = info_player.loc[0, 'time_eliminated']
    df_master.loc[i, 'total_damage_to_players'] = info_player.loc[0, 'total_damage_to_players']

In [115]:
df_master

,match_id,gold_left,last_round,level,placement,players_eliminated,puuid,time_eliminated,total_damage_to_players,blademaster_num,...,champion_1,champion_2,champion_3,champion_4,champion_5,champion_6,champion_7,champion_8,champion_9,champion_10
0,BR1_1926730533,0,27,7,6,0,qYjGRXS83Z7LIC4c40Hk27k2ee6SrYKWhGOwdRbz2IjXf4...,1514.83,32,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BR1_1926730533,2,40,8,1,4,ZOp977RUzO2rc4XXpC5LDmSW9FVPinsos54x01UvxLH98P...,2191.27,242,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,BR1_1926730533,0,34,7,3,0,BsmmhSCiK5vK67zbbMhAz2edViDJ0dvNlTuQOZQNcbu13x...,1923.12,86,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BR1_1926730533,0,27,7,7,1,Nd0RYeDsIefGnmlGA45x6zj-thg349ozYFcEobPFfROdqd...,1511.44,43,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,BR1_1926730533,0,33,7,5,0,VZV01cG8R-A-yZ5spGtEhMTOfsdVBZhf-4EDIkcEhgjb3M...,1857.45,62,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,BR1_1926730533,1,40,9,2,2,K3mbUvb-2PPBom5fdP5FBdn-2CUsQqy4t2wh-dmw-e55tv...,2191.27,158,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,BR1_1926730533,18,34,7,4,0,yWg9adnTaYHLhXpyOgkDilt9Oyv80_n4-RlL19ZpQtnXpO...,1918.49,107,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,BR1_1926730533,1,26,6,8,0,OpAqJOjE1o8Y1l3cb50mh1iaUnDONPCzEZUm8fobWVot-6...,1447.23,42,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [117]:
info_player.iloc[0, -2]

[{'name': 'Chrono',
  'num_units': 2,
  'style': 1,
  'tier_current': 1,
  'tier_total': 3},
 {'name': 'DarkStar',
  'num_units': 1,
  'style': 0,
  'tier_current': 0,
  'tier_total': 3},
 {'name': 'Rebel',
  'num_units': 3,
  'style': 1,
  'tier_current': 1,
  'tier_total': 3},
 {'name': 'Set3_Blademaster',
  'num_units': 3,
  'style': 1,
  'tier_current': 1,
  'tier_total': 3},
 {'name': 'Set3_Brawler',
  'num_units': 1,
  'style': 0,
  'tier_current': 0,
  'tier_total': 2},
 {'name': 'Set3_Mystic',
  'num_units': 2,
  'style': 1,
  'tier_current': 1,
  'tier_total': 2}]

In [88]:
info_player1 = pd.DataFrame([dict_player1[7]])
info_player1

,companion,gold_left,last_round,level,placement,players_eliminated,puuid,time_eliminated,total_damage_to_players,traits,units
0,{'content_ID': 'f56ecac7-ed16-483d-8b9c-2bbced...,1,26,6,8,0,OpAqJOjE1o8Y1l3cb50mh1iaUnDONPCzEZUm8fobWVot-6...,1447.231934,42,"[{'name': 'Chrono', 'num_units': 2, 'style': 1...","[{'character_id': 'TFT3_Yasuo', 'items': [14, ..."


In [89]:
pd.DataFrame([info_player1.iloc[0,-2]])

,0,1,2,3,4,5
0,"{'name': 'Chrono', 'num_units': 2, 'style': 1,...","{'name': 'DarkStar', 'num_units': 1, 'style': ...","{'name': 'Rebel', 'num_units': 3, 'style': 1, ...","{'name': 'Set3_Blademaster', 'num_units': 3, '...","{'name': 'Set3_Brawler', 'num_units': 1, 'styl...","{'name': 'Set3_Mystic', 'num_units': 2, 'style..."


In [49]:
pd.DataFrame([{'gold_left':0, 'last_round': pd.NA}])

,gold_left,last_round
0,0,<NA>


In [104]:


df = pd.DataFrame(columns = ['match_id', 'gold_left', 'last_round', 'level', 'placement', 'players_eliminated', 'puuid',
                             'time_eliminated', 'total_damage_to_players', 'blademaster_num', 'blaster_num',
                             'brawler_num', 'celestial_num', 'chrono_num', 'cybernetic_num', 'dark_star_num',
                             'demolitionist_num', 'infiltrator_num', 'mana_reaver_num', 'mech_pilot_num',
                             'mercenary_num', 'mystic_num', 'protector_num', 'rebel_num', 'sniper_num',
                             'space_pirate_num', 'star_guardian_num', 'starship_num', 'sorcerer_num', 
                             'valkyrie_num', 'vanguard_num', 'void_num', 'champion_1', 'champion_2', 'champion_3',
                             'champion_4', 'champion_5', 'champion_6', 'champion_7', 'champion_8', 'champion_9',
                             'champion_10'])


In [105]:
df.loc[6, 'level'] = 7

In [106]:
df

,match_id,gold_left,last_round,level,placement,players_eliminated,puuid,time_eliminated,total_damage_to_players,blademaster_num,...,champion_1,champion_2,champion_3,champion_4,champion_5,champion_6,champion_7,champion_8,champion_9,champion_10
6,NaN,NaN,NaN,7,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
